In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
url = "../datasets/DummyLogistic/Data-L-Reg-Gradient-Descent.csv"

# load dataset into Pandas DataFrame
df = pd.read_csv(url, na_values= ['?'])
features = df.columns

print("Non estandarized data")
df.head()

Non estandarized data


,x1,x2,y,X1plusX2
0,89.946963,72.091781,1,162.038744
1,77.111425,75.206589,1,152.318014
2,71.854053,80.549965,1,152.404018
3,67.182350,97.026782,1,164.209131
4,71.087755,71.098405,1,142.186160


In [3]:
df.shape

(400, 4)

In [4]:
X = df.as_matrix()[:, 0:-2]
Y = df.as_matrix()[:, -2]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [6]:
Y.shape

(400,)

In [7]:
X_train = X[:350, :]
Y_train = Y[:350].astype(int).reshape((350,1))
X_test = X[350:, :]
Y_test = Y[350:].astype(int).reshape((50,1))

In [8]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((350, 2), (350, 1), (50, 2), (50, 1))

In [9]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [10]:
def logisticRegression(X, theta):
    return sigmoid(X@theta)

In [11]:
def costFunction(X, Y, theta, lam):
    z = X@theta
    h = sigmoid(z)
    return (1/X.shape[0])*(-Y.T@np.log(h)-(1-Y).T@np.log(1-h)+(lam/2)*theta.T@theta)

In [12]:
def gradient(X, y, theta, lr, lam):
    z = np.dot(X, theta)
    h = sigmoid(z)
    new_theta = theta
    new_theta[0] = 0
    gradient = (np.dot(X.T, (h - y))+lam*new_theta) / y.size
    return theta - lr * gradient

In [13]:
lr = 0.03
lam = 0

In [14]:
theta = np.zeros((X_train.shape[1], 1))

In [15]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros((X.shape[1], 1))
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            loss = self.__loss(h, y)
                
            if(self.verbose ==True and i % 10000 == 0):
                print(f'loss: {loss} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X):
        return self.predict_prob(X).round()

In [16]:
model = LogisticRegression(lr=0.03, num_iter=1000, fit_intercept=False)

In [17]:
model.fit(X_train, Y_train)

In [18]:
preds = model.predict(X_test)
boolarr = preds == Y_test

In [21]:
np.sum(boolarr)/boolarr.size

0.78

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
logistic = LogisticRegression()
logistic.fit(X_train, Y_train.reshape(Y_train.shape[0]))
pred = logistic.predict(X_test)

In [24]:
boolarr = pred == Y_test
np.sum(boolarr)/boolarr.size

0.5016

In [25]:
for i in range(100000):
    theta = gradient(X_train, Y_train, theta, lr, lam)

print("Train: ", costFunction(X_train, Y_train, theta, lam))
print("Test: ", costFunction(X_test, Y_test, theta, lam))

Train:  [[0.5311149]]
Test:  [[0.52202349]]


In [26]:
predic = logisticRegression(theta=theta, X=X_test)

In [27]:
boolarr = np.round(predic) == Y_test
np.sum(boolarr)/boolarr.size

0.7

In [28]:
theta, model.theta

(array([[0.00483517],
        [1.46126234]]), array([[1.03657748],
        [1.56109163]]))

In [29]:
costFunction(theta=theta, X=X_test, Y=Y_test, lam=lam)

array([[0.52202349]])

In [30]:
X_train_ = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
Y_train_ = Y_train
X_test_ = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)
Y_test_ = Y_test

In [31]:
thetat = np.zeros((X_train_.shape[1], 1))

In [32]:
for i in range(100000):
    thetat = gradient(X_train_, Y_train_, thetat, lr, lam)

print("Train: ", costFunction(X_train_, Y_train_, thetat, lam))
print("Test: ", costFunction(X_test_, Y_test_, thetat, lam))

Train:  [[0.44895881]]
Test:  [[0.4254783]]


In [33]:
predic = logisticRegression(theta=thetat, X=X_test_)

In [34]:
boolarr = np.round(predic) == Y_test_
np.sum(boolarr)/boolarr.size

0.78

In [36]:
theta, thetat, model.theta

(array([[0.00483517],
        [1.46126234]]), array([[-1.55557182e-04],
        [ 1.12689498e+00],
        [ 1.69316958e+00]]), array([[1.03657748],
        [1.56109163]]))